<a href="https://colab.research.google.com/github/tasniaishrar/Movie-Recommendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('movies_metadata.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# Formula used by IMDB for weighted rating of each movie
 **(v/(v+m) * R) + (m/(m+v) * C), where**


*   **v** ---> number of votes for the movie
*   **m** ---> min votes required to be listed on the chart


*   **R** ---> avergae rating of the movie
*   **C** ---> mean vote across all movies





In [2]:
#calculate C, the mean vote across the whole report
C = df['vote_average'].mean()
print(C)


5.618207215133889


In [3]:
#min votes required to be on the chart, m
m= 25000
#filter out all qualified movies into a new DataFrame
qualified = df.copy().loc[df['vote_count'] >= m]
qualified

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [4]:
#in this dataset, no movie has a vote count above 25,000 as set by IMDb so we can just take 90th percentile vote counts as min 
m = df['vote_count'].quantile(0.90)
print(m)


160.0


In [5]:
#all qualified movies into a new DataFrame
qualified = df.copy().loc[df['vote_count'] >= m]
qualified

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45177,False,"{'id': 442352, 'name': 'Brice Collection', 'po...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,375798,tt5029602,fr,Brice 3,"Brice is back. The world has changed, but not ...",7.276407,/d42seAYATQhh3y4GxFQkM46jHvq.jpg,"[{'name': 'Gaumont', 'id': 9}, {'name': 'M6 Fi...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2016-10-19,0.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Brice 3,False,4.3,160.0
45204,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,417870,tt3564472,en,Girls Trip,Four girlfriends take a trip to New Orleans fo...,37.964872,/94nCLn74lBh4pLdZPIad18BpheE.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-07-21,0.0,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"""Forgive us in advance for this wild weekend""",Girls Trip,False,7.1,393.0
45258,False,"{'id': 466463, 'name': 'Descendants Collection...",0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 107...",NaN,417320,tt5117876,en,Descendants 2,When the pressure to be royal becomes too much...,15.842073,/7TLywOVzWwU1GMyqdjUiSVzBiyc.jpg,"[{'name': 'Walt Disney Television', 'id': 670}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-07-21,0.0,111.0,"[{'iso_639_1': 'da', 'name': 'Dansk'}]",Released,Long live evil.,Descendan

In [6]:
#function for computing the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count'] #no of votes for that movie
    R = x['vote_average'] #avg rating for the movie
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [7]:
#new column 'score' and updating its value with `weighted_rating()`
#axis=1 bc we want to apply the function to the df column instead of index
qualified['score'] = qualified.apply(weighted_rating,axis=1)
qualified[['original_title','score']]

,original_title,score
0,Toy Story,7.640253
1,Jumanji,6.820293
4,Father of the Bride Part II,5.660700
5,Heat,7.537201
8,Sudden Death,5.556626
...,...,...
45177,Brice 3,4.959104
45204,Girls Trip,6.671272
45258,Descendants 2,6.590372
45265,Turist,6.344369


In [8]:
#sort movies in descending order based on score 
qualified = qualified.sort_values('score', ascending=False)

#display top 250
#A simple clone of IMDB Top 250 Chart
qualified[['title', 'vote_count', 'vote_average', 'score']].head(250)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
...,...,...,...,...
42015,The Invisible Guest,395.0,8.1,7.384528
1233,Young Frankenstein,880.0,7.7,7.379724
24902,Wild Tales,878.0,7.7,7.379107
1054,Dial M for Murder,539.0,7.9,7.377701


## Content based recommendation
This recommends movies that are similar to a particular movie. A pairwise cosine similarity scores for all movies is computed based on their plot descriptions and recommend movies based on that similarity score threshold.

In [9]:
#overviews of the first 5 movies
df['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

not possible to compute the similarity between any two overviews in their raw forms so need to compute the word vectors (carrying semantic meanings)of each overview 
For example, woman and queen will have similar vector representations while woman and man will be different

In [10]:
#TfIdfVectorizer: Term Frequency-Inverse Document Frequency
#gives matrix where each column represents a word in the overview vocabulary and each row represents a movie
from sklearn.feature_extraction.text import TfidfVectorizer

#remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
#taking first 30000 rows since colab crashes when all rows used for linear_kernel
tfidf_matrix = tfidf.fit_transform(df['overview'].head(30000))

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(30000, 58562)

In [11]:
#from above output, 30000 different words in the dataset have 58562 movies
#array mapping from feature integer indices to feature name
tfidf.get_feature_names()[2990:3000]

['aramoana',
 'aran',
 'arano',
 'arapaho',
 'arashi',
 'arasındaki',
 'arata',
 'aravidze',
 'araya',
 'araújo']

In [12]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix: calculating the dot product between each vector 
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(30000, 30000)

a matrix of shape 30000x30000, which means each movie overview cosine similarity score with every other movie overview. Hence, each movie is a 1x3000 column vector where each column is a similarity score with each movie

In [13]:
#Construct a reverse map of indices and movie titles, and remove duplicates 
indices = pd.Series(df.head(30000).index, index=df['title'].head(30000)).drop_duplicates()
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [76]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    #index of the movie that matches the title
    idx = indices[title]

    #pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    if len(sim_scores)>=10:
      #sort the movies based on the similarity scores and get 10 most similar
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
    else:
      #sort them and display the only results produced 
      sim_scores = sorted(sim_scores, reverse=True)

    #get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    #display most similar movies (top 10 if more than 10 results)
    return df['title'].iloc[movie_indices]

In [69]:
get_recommendations('The Shawshank Redemption')

16947                   They Made Me a Fugitive
6548                                Civil Brand
11327                               Brute Force
17446                           Girls in Prison
9391                                    In Hell
9225     Female Prisoner Scorpion: Jailhouse 41
29422                      The Domino Principle
28512                     Letters From a Killer
29425                                      Riot
8307                              Sherlock, Jr.
Name: title, dtype: object

In [77]:
get_recommendations('The Avengers')

1      Jumanji
0    Toy Story
Name: title, dtype: object

In [73]:
get_recommendations('Toy Story')

15348                    Toy Story 3
2997                     Toy Story 2
10301         The 40 Year Old Virgin
24523                      Small Fry
23843    Andy Hardy's Blonde Trouble
29202                     Hot Splash
8327                       The Champ
27206     Life Begins for Andy Hardy
1071           Rebel Without a Cause
26304         You're Only Young Once
Name: title, dtype: object

In [74]:
get_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [75]:
get_recommendations('Batman Returns')

21194    Batman Unmasked: The Psychology of the Dark Kn...
15511                           Batman: Under the Red Hood
18252                                The Dark Knight Rises
25267                                    Batman vs Dracula
20232              Batman: The Dark Knight Returns, Part 2
12481                                      The Dark Knight
3095                          Batman: Mask of the Phantasm
23878                            Batman: Assault on Arkham
21124       Lego Batman: The Movie - DC Super Heroes Unite
29536                   Batman Unlimited: Animal Instincts
Name: title, dtype: object